**Please run the above line to refresh the date before your submission.**


## Data Structures - Recitation 10

### Map, Dictionary, and Hash Table

Submit your worksheet according to the schedule for your recitation as outlined on Albert:

- **Wednesday Recitation:** Due by Friday, 11:59 PM.
- **Thursday Recitation:** Due by Saturday, 11:59 PM.
- **Friday Recitation:** Due by Sunday, 11:59 PM.


**Important Notes**
- Each task must show a reasonable attempt to a solution. 
- Only solutions submitted in *.ipynb* format are accepted.
- Invalid and late submissions are not considered for grading. 

Name:Ziyue Xu

NetID:zx1790

# Learning Outcomes

In this recitation, you will learn about the following topics:
1. **???:** ???
2. **???:** ???
3. **???:** ???
4. **???:** ???

# Given Class - MapBase

In [ ]:
from collections.abc import MutableMapping


class MapBase(MutableMapping):
    """
    An abstract base class that includes a nonpublic _Item class.
    """
    
    #------------------------------- nested _Item class -------------------------------

    class _Item:
        """
        Lightweight composite to store key-value pairs as map items.
        """
        __slots__ = '_key', '_value'

        def __init__(self, k, v):
            """
            Initializer for a key-value pair.
            
            :param k: The key of the item.
            :param v: The value of the item.
            """
            self._key = k
            self._value = v

        def __eq__(self, other):
            """
            Magic method to compare two keys for equality.
            
            :param other: Another _Item object to compare.
            :return: True if the key of self is equal to the key of other, False otherwise.
            """
            return self._key == other._key

        def __ne__(self, other):
            """
            Magic method to compare two keys for inequality.
            
            :param other: Another _Item object to compare.
            :return: True if the key of self is not equal to the key of other, False otherwise.
            """
            return not (self == other)

        def __lt__(self, other):
            """
            Magic method to compare two keys by size.
            
            :param other: Another _Item object to compare.
            :return: True if the key of self is less than the key of other, False otherwise.
            """
            return self._key < other._key

# Given Class - UnsortedTableMap

In [ ]:
class UnsortedTableMap(MapBase):
    """Map implementation using an unordered list."""

    def __init__(self):
        """Create an empty map."""
        self._table = []

    def __getitem__(self, k):
        """Return value associated with key k (raise KeyError if not found)."""
        for item in self._table:
            if k == item._key:
                return item._value
        raise KeyError('Key Error: ' + repr(k))

    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present."""
        for item in self._table:
            if k == item._key:
                item._value = v
                return
        self._table.append(self._Item(k, v))

    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)."""
        for j in range(len(self._table)):
            if k == self._table[j]._key:
                self._table.pop(j)
                return
        raise KeyError('Key Error: ' + repr(k))

    def __len__(self):
        """Return number of items in the map."""
        return len(self._table)

    def __iter__(self):
        """Generate iteration of the map's keys."""
        for item in self._table:
            yield item._key

# Given Class - HashMapBase

In [ ]:
from random import randrange


class HashMapBase(MapBase):
    """
    Abstract base class for a map using hash-table with MAD compression.
    MAD - Multiply-Add-Divide, is a compression function that will spread 
    integer (more) evenly over the range [0..(N-1)] if we use a prime number for p.
    """

    def __init__(self, cap=11, p=109345121):
        """
        Create an empty hash-table map.
        
        :param cap: Initial table size (default 11)
        :param p: Positive prime used for MAD (default 109345121) 
        """
        self._table = cap * [None]
        self._n = 0
        self._prime = p
        self._scale = 1 + randrange(p - 1)
        self._shift = randrange(p)

    def _hash_function(self, k):
        """
        The hash function for mapping keys to table indices.
        
        :param k: The key to hash. 
        :return: The hash value of the key.
        """
        return (hash(k) * self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        """
        Magic method to get the length of the hash table.
        
        :return: The number of items in the hash table.
        """
        return self._n

    def __getitem__(self, k):
        """
        Magic method to get an item from the hash table.
        
        :param k: The key to get.
        :return: The value associated with the key.
        """
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)

    def __setitem__(self, k, v):
        """
        Magic method to set an item in the hash table.
        The method also checks if the table needs to be resized.
        
        :param k: The key to set.
        :param v: The value to set.
        :return: None. Nothing is returned.
        """
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)
        if self._n > len(self._table) // 2:
            self._resize(2 * len(self._table) - 1)

    def __delitem__(self, k):
        """
        Magic method to delete an item from the hash table.
        The method raises a KeyError if the key is not found.
        
        :param k: The key to delete.
        :return: None. Nothing is returned.
        """
        j = self._hash_function(k)
        self._bucket_delitem(j, k)
        self._n -= 1

    def _resize(self, c):
        """
        Resize the hash table to capacity c and rehash all items.
        
        :param c: The new capacity of the hash table.
        :return: None. Nothing is returned.
        """
        old = list(self.items())
        self._table = c * [None]
        self._n = 0
        for (k, v) in old:
            self[k] = v

# Given Class - ChainHashMap

In [ ]:
class ChainHashMap(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution."""

    def _bucket_getitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))
        return bucket[k]

    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap()
        old_size = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > old_size:
            self._n += 1

    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))
        del bucket[k]

    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:
                for key in bucket:
                    yield key

# Given Class - ProbeHashMap

In [ ]:
class ProbeHashMap(HashMapBase):
    """Hash map implemented with linear probing for collision resolution."""
    _AVAIL = object()

    def _is_available(self, j):
        """Return True if index j is available in table."""
        return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL

    def _find_slot(self, j, k):
        """
        Search for key k in bucket at index j.
        
        :param j: The index of the bucket
        :param k: The key to search
        :return: Returns a tuple of (success, index) 
        """
        firstAvail = None
        while True:
            if self._is_available(j):
                if firstAvail is None:
                    firstAvail = j
                if self._table[j] is None:
                    return False, firstAvail
            elif k == self._table[j]._key:
                return True, j
            j = (j + 1) % len(self._table)

    def _bucket_getitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError('Key Error: ' + repr(k))
        return self._table[s]._value

    def _bucket_setitem(self, j, k, v):
        found, s = self._find_slot(j, k)
        if not found:
            self._table[s] = self._Item(k, v)
            self._n += 1
        else:
            self._table[s]._value = v

    def _bucket_delitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError('Key Error: ' + repr(k))
        self._table[s] = ProbeHashMap._AVAIL

    def __iter__(self):
        for j in range(len(self._table)):
            if not self._is_available(j):
                yield self._table[j]._key

    def __str__(self):
        result = []
        result.append("[")
        for j in range(len(self._table)):
            if not self._is_available(j):
                result.append("(" + str(self._table[j]._key) + " " + str(self._table[j]._value) + "), ")
            else:
                result.append("None, ")
        result.append("]")
        return "".join(result)


def main():
    table = ProbeHashMap()
    values = ["Ezreal", "Blizcrank", "Annie", "Teemo", "Zed"]
    for i in range(len(values)):
        table[i] = values[i]

    print(table)


if __name__ == '__main__':
    main()

# Task 1 - Modifying HashMapBase

Modify the class *HashMapBase(MapBase)*. The HashMapBase maintains a load factor λ = 0.5. Can you locate this code? Modify load factor λ to 0.66.


In [ ]:
class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression.

    ### Note: MAD: Multiply-Add-Divide
    From Mathematical analysis (group theory), this compression function will spread integer (more) evenly over the range [0..(N-1)] if we use a prime number for p.

    Keys must be hashable and non-None.
    """

    def __init__(self, cap=11, p=109345121):
        """
        Create an empty hash-table map.
        
        :param cap: Initial table size (default 11)
        :param p: Positive prime used for MAD (default 109345121) 
        """
        self._table = cap * [None]
        self._n = 0
        self._prime = p
        self._scale = 1 + randrange(p - 1)
        self._shift = randrange(p)

    def _hash_function(self, k):
        return (hash(k) * self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)

    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)
        if self._n > len(self._table) * 0.66:
            self._resize(2 * len(self._table) - 1)

    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)
        self._n -= 1

    def _resize(self, c):
        """Resize bucket array to capacity c and rehash all items."""
        old = list(self.items())
        self._table = c * [None]
        self._n = 0
        for (k, v) in old:
            self[k] = v

# Task 2 - Modify ProbeHashMap

Modify the class *ProbeHashMap(HashMapBase)*. This file currently uses linear probing to handle collisions. After modification, your hashing should use quadratic probing instead of linear probing.

In [ ]:
class ProbeHashMap(HashMapBase):
    """Hash map implemented with linear probing for collision resolution."""
    _AVAIL = object()

    def _is_available(self, j):
        """Return True if index j is available in table."""
        return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL

    def _find_slot(self, j, k):
        """
        Search for key k in bucket at index j.
        
        :param j: The index of the bucket
        :param k: The key to search
        :return: Returns a tuple of (success, index) 
        """
        firstAvail = None
        i=1
        while True:
            if self._is_available(j):
                if firstAvail is None:
                    firstAvail = j
                if self._table[j] is None:
                    return False, firstAvail
            elif k == self._table[j]._key:
                return True, j
            j = (j + i**2) % len(self._table)
            i+=1


    def _bucket_getitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError('Key Error: ' + repr(k))
        return self._table[s]._value

    def _bucket_setitem(self, j, k, v):
        found, s = self._find_slot(j, k)
        if not found:
            self._table[s] = self._Item(k, v)
            self._n += 1
        else:
            self._table[s]._value = v

    def _bucket_delitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError('Key Error: ' + repr(k))
        self._table[s] = ProbeHashMap._AVAIL

    def __iter__(self):
        for j in range(len(self._table)):
            if not self._is_available(j):
                yield self._table[j]._key

    def __str__(self):
        result = []
        result.append("[")
        for j in range(len(self._table)):
            if not self._is_available(j):
                result.append("(" + str(self._table[j]._key) + " " + str(self._table[j]._value) + "), ")
            else:
                result.append("None, ")
        result.append("]")
        return "".join(result)


def main():
    table = ProbeHashMap()
    values = ["Ezreal", "Blizcrank", "Annie", "Teemo", "Zed"]
    for i in range(len(values)):
        table[i] = values[i]

    print(table)


if __name__ == '__main__':
    main()

# Task 3 - Modify HashMapBase

Modify the class *HashMapBase(MapBase)*. In the class *HashMapBase(MapBase)*, find out the hash function, what type of compression method is it using? Change the compression method using Division method. Recall Division method: Index = k % N

In [ ]:
class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression.

    ### Note: MAD: Multiply-Add-Divide
    From Mathematical analysis (group theory), this compression function will spread integer (more) evenly over the range [0..(N-1)] if we use a prime number for p.

    Keys must be hashable and non-None.
    """

    def __init__(self, cap=11, p=109345121):
        """
        Create an empty hash-table map.
        
        :param cap: Initial table size (default 11)
        :param p: Positive prime used for MAD (default 109345121) 
        """
        self._table = cap * [None]
        self._n = 0
        self._prime = p
        self._scale = 1 + randrange(p - 1)
        self._shift = randrange(p)

    def _hash_function(self, k):
        return hash(k) % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)

    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)
        if self._n > len(self._table) // 2:
            self._resize(2 * len(self._table) - 1)

    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)
        self._n -= 1

    def _resize(self, c):
        """Resize bucket array to capacity c and rehash all items."""
        old = list(self.items())
        self._table = c * [None]
        self._n = 0
        for (k, v) in old:
            self[k] = v

# Task 4 - Contains

Implement the function *l1_contains_l2(l1, l2)* below. This function checks if every element in l2 exists in l1. Use *ProbeHashMap(Linear probing HashMap)* to solve this problem. Assume all elements in l1, l2 are unique.

In [ ]:
def l1_contains_l2(l1, l2):
    """
    Checks if every element in l2 exists in l1.
    
    :param l1: A python list with immutable elements.
    :param l2: A python list with immutable elements.
    :return: True if all elements in l2 exist in l1, false otherwise.
    """

    # Please write your code here.
    hashtable=ProbeHashMap()
    for i in l1:
        hashtable[i]=i
    print(hashtable)
    for j in l2:
        try:
            temp=hashtable[j]
            if temp!=j:
                return False
        except:
            return False
    return True


def main():
    l1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
    l2 = [5, 2, 8, 9, 0, 1]
    l3 = [5, 2, 8, 9, 0, 1, "haha"]
    assert l1_contains_l2(l1, l2) == True, "l2 should be a subset of l1 but was not found so by l1_contains_l2"
    assert l1_contains_l2(l1, l3) == False, "l3 should not be a subset of l1 but was found so by l1_contains_l2"

    print("All tests passed successfully!")


if __name__ == '__main__':
    main()

# Task 5 - Count Words

Complete the following code. This program counts words in a given text file, then output the most frequent word and its frequency.


In [ ]:
def main():

    table = ProbeHashMap()
    file = open("count_words.txt", "r")

    # Please write your code here.
    lines=file.readlines()
    max_word = ""
    max_count = 0
    for line in lines:
        words=line.split()
        for word in words:
            try:
                table[word]+=1
                if table[word]>max_count:
                    max_word=word
                    max_count=table[word]
            except:
                table[word]=1
                if table[word]>max_count:
                    max_word=word
                    max_count=table[word]

    print('The most frequent word is', max_word)
    print('Its number of occurrences is', max_count)


if __name__ == '__main__':
    main()

# Task 6 and Task 7 - Union and Intersection

Given two python lists, create union and intersection lists that contain union and intersection of the elements present in the given lists. Order of elements in output lists doesn't matter.

1. Implement function *union(l1, l2)*. This function returns a new list, that is the union of l1 and l2. Use Python dictionary to reduce runtime. Assume all elements in l1, l2 are unique.
2. Implement function *intersection(l1, l2)*. This function returns a new list, that is the intersection of l1 and l2. Use Python dictionary to reduce runtime. Assume all elements in l1, l2 are unique.

In [ ]:
def union(l1, l2):
    """
    Creates a union of two lists.
    
    :param l1: A python list with immutable elements.
    :param l2: A python list with immutable elements.
    :return: A new list that is the union of l1 and l2.
    """

    # Please write your code here.
    answer=l1[:]
    dic1={}
    for i in l1:
        dic1[i]=True
    for j in l2:
        try:
            if not dic1[j]:
                answer.append(j)
        except:
            continue
    return answer



def intersection(l1, l2):
    """
    Creates the intersection of two lists.
    
    :param l1: A python list with immutable elements.
    :param l2: A python list with immutable elements.
    :return: A new list that is the intersection of l1 and l2.
    """

    # Please write your code here.
    answer=[]
    dic1={}
    for i in l1:
        dic1[i]=True
    for j in l2:
        try:
            if dic1[j]:
                answer.append(j)
        except:
            continue
    return answer


def main():
    l1 = [10, 15, 4, 20]
    l2 = [8, 4, 2, 10]

    union_result = union(l1, l2)
    intersection_result = intersection(l1, l2)

    assert set(union_result) == {10, 15, 4, 20, 8, 2}, "Error in union function"
    assert set(intersection_result) == {4, 10}, "Error in intersection function"


if __name__ == '__main__':
    main()